<a href="https://colab.research.google.com/github/onticonti/Face_Detection_Web_App/blob/master/Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install pyngrok
!pip install opencv-python

     |████████████████████████████████| 7.2MB 3.5MB/s 
     |████████████████████████████████| 4.4MB 39.9MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 112kB 38.9MB/s 
     |████████████████████████████████| 122kB 46.1MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=25fc5c11e8ff379f6e4bcc12b5013121a0112d8e8769ea6962bba09d28468ff7
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=3ba20c24ea1e6b6574a0ab12dd5b16a9bf9d4fd35a01c71c602c75bc3c00eefa
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=93cfff3dd74d8141c47c32d05ae03ce7bd433dbf532c2176922f5c026282a32b
  Stored in directory: /root/.cache/pip/whee

  Created wheel for pyngrok: filename=pyngrok-4.1.12-cp36-none-any.whl size=16810 sha256=5f79e51eab8b728ce719ddc61be50e2c52d9dddc5da8c7988a8ff6d86622c103
  Stored in directory: /root/.cache/pip/wheels/69/c3/d6/6968dd4d831794d41c311be1d7af6f4ac151c5d3bd0e6efab8
Successfully built pyngrok


In [21]:
%%writefile app.py

import streamlit as st
import cv2
from PIL import Image,ImageEnhance
import numpy as np
import os
from io import StringIO

@st.cache
def load_image(img):
  im = Image.open(img)
  return im

face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/content/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('/content/haarcascade_smile.xml')

def detect_faces(our_image):
  new_img = np.array(our_image.convert('RGB'))
  img = cv2.cvtColor(new_img,1)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  faces = face_cascade.detectMultiScale(gray,1.1,4)
  #Draw Rectangle
  for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  return img,faces

def detect_eyes(our_image):
  new_img = np.array(our_image.convert('RGB'))
  img = cv2.cvtColor(new_img,1)
  gray = cv2.cvtColor(new_img,cv2.COLOR_BGR2GRAY)
  eyes = eye_cascade.detectMultiScale(gray,1.3,5)
  for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

  return img
def detect_smiles(our_image):

	new_img = np.array(our_image.convert('RGB'))
	img = cv2.cvtColor(new_img,1)
	gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
	# Detect Smiles
	smiles = smile_cascade.detectMultiScale(gray, 1.1, 4)
	# Draw rectangle around the Smiles
	for (x, y, w, h) in smiles:
	  cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
	return img

def cartonize_image(our_image):
	new_img = np.array(our_image.convert('RGB'))
	img = cv2.cvtColor(new_img,1)
	gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
	# Edges
	gray = cv2.medianBlur(gray, 5)
	edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
	#Color
	color = cv2.bilateralFilter(img, 9, 300, 300)
	#Cartoon
	cartoon = cv2.bitwise_and(color, color, mask=edges)

	return cartoon


def cannize_image(our_image):
	new_img = np.array(our_image.convert('RGB'))
	img = cv2.cvtColor(new_img,1)
	img = cv2.GaussianBlur(img, (11, 11), 0)
	canny = cv2.Canny(img, 100, 150)
	return canny


def main():

  
  st.title("Face Detection App")
  st.text("With Streamlit")

  activities = ["Detection","About"]
  choice = st.sidebar.selectbox("Select Activty",activities)


  if choice == 'Detection':

    st.subheader("Face Detection")
    st.set_option('deprecation.showfileUploaderEncoding', False)
    image_file = st.file_uploader("Upload your Image",type=['jpg','png','jpeg'])


    if image_file is not None:
      our_image = Image.open(image_file)
      st.text("Original Image")
      st.write(type(our_image))
      st.image(our_image)

    
    enhance_type = st.sidebar.radio("Enhance Type",["Original","Gray-Scale","Contrast","Brightness","Blurring"])
    if enhance_type == 'Gray-Scale':
      new_img = np.array(our_image.convert('RGB'))
      img = cv2.cvtColor(new_img,1)
      gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      #st.write(new_img) ---> np.array
      st.image(gray)

    if enhance_type == 'Contrast':
      c_rate = st.sidebar.slider("Contrast",0.5,3.5)
      enhancer = ImageEnhance.Contrast(our_image)
      img_output = enhancer.enhance(c_rate)
      st.image(img_output)

    if enhance_type == 'Brightness':
      c_rate = st.sidebar.slider("Brightness",0.5,3.5)
      enhancer = ImageEnhance.Brightness(our_image)
      img_output = enhancer.enhance(c_rate)
      st.image(img_output)

    if enhance_type == 'Blurring':
      new_img = np.array(our_image.convert('RGB'))
      blur_rate = st.sidebar.slider("Blurring",0.5,3.5)
      img = cv2.cvtColor(new_img,1)
      blur_img = cv2.GaussianBlur(img,(11,11),blur_rate)
      st.image(blur_img)


    #Face Detection

    task = ["Faces","Smiles","Eyes","Cannize","Cartonize"]
    feature_choice = st.sidebar.selectbox("Find Features",task)

    if st.button("Process"):

      if feature_choice == 'Faces':
        result_img,result_faces = detect_faces(our_image)
        st.image(result_img)
        st.success("Found {} faces".format(len(result_faces)))

      elif feature_choice == 'Smiles':
        result_img = detect_smiles(our_image)
        st.image(result_img)

      elif feature_choice == 'Eyes':
        result_img = detect_eyes(our_image)
        st.image(result_img)

      elif feature_choice == 'Cartonize':
        result_img = cartonize_image(our_image)
        st.image(result_img)

      elif feature_choice == 'Cannize':
        result_canny = cannize_image(our_image)
        st.image(result_canny)
      
  elif choice == 'About':
    st.subheader("About Opencv2")
    st.write("https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_setup/py_intro/py_intro.html#intro")
    

if __name__ == '__main__':
  main()


Overwriting app.py


In [3]:
!ls

app.py		     haarcascade_frontalface_default.xml  sample_data
haarcascade_eye.xml  haarcascade_smile.xml


In [5]:
!/ngrok authtoken 1hQOLnMvFOe3mO0fiAWkdsdE9HU_3XqipxFmRgUicr8Enpgg7


/bin/bash: /ngrok: No such file or directory


In [6]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [7]:
from pyngrok import ngrok

In [8]:
#!nohub streamlit run app.py
!streamlit run app.py&>/dev/null&

In [9]:
!pgrep streamlit

291


In [10]:
pub_url = ngrok.connect(port='8501')

In [11]:
pub_url

'http://a5db53818604.ngrok.io'

In [ ]:
#Shutdown 
!kill 445

/bin/bash: line 0: kill: (445) - No such process
